In [1]:
import gpxpy
import numpy as np
import pandas as pd
from kompy import KomootConnector
import os
connector = KomootConnector(
    email=os.environ['KOMOOT_EMAIL'],
    password=os.environ['KOMOOT_PASSWORD'],
)
gpx_track = connector.get_tour_by_id(tour_identifier='1378844177', object_type='gpx')


In [2]:
gpx_track

GPX(tracks=[GPXTrack(name='W02D2-Recovery Run', segments=[GPXTrackSegment(points=[...])])])

In [3]:
from gpxpy.gpx import GPXTrackPoint
import math
from geopy import distance

def calculate_initial_compass_bearing(first_point: GPXTrackPoint, second_point: GPXTrackPoint):
    """
    Calculates the bearing between two points.

    The formula used to calculate the bearing is:
        θ = atan2(sin(Δlong).cos(lat2), cos(lat1).sin(lat2) - sin(lat1).cos(lat2).cos(Δlong))

    :param first_point: The first point
    :param second_point: The second point
    :return: The bearing between the two points in degrees
    """

    lat1 = math.radians(first_point.latitude)
    lat2 = math.radians(second_point.latitude)
    diffLong = math.radians(second_point.longitude - first_point.longitude)

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Normalize the initial bearing
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

def calculate_turn_angle(bearing1, bearing2):
    """
    Calculate the angle of turn between two bearings.
    """
    angle = bearing2 - bearing1
    
    angle = (angle + 180) % 360 - 180

    return angle




def load_gpx_todf(gpx_data):
    gpx_points = []
    for track in gpx_data.tracks:
        for segment in track.segments:
            for point in segment.points:
                gpx_points.append(
                    {
                        'gpx_point': point,
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "elevation": point.elevation,
                        "time": point.time,
                        "distance": distance.geodesic(
                            (point.latitude, point.longitude),
                            (gpx_points[-1]['latitude'],gpx_points[-1]['longitude']),
                        ).m if len(gpx_points) > 0 else 0,
                        'time_diff_s': (point.time - gpx_points[-1]['gpx_point'].time).total_seconds() if len(gpx_points) > 0 else 0,
                        'elevation_diff': point.elevation - gpx_points[-1]['gpx_point'].elevation if len(gpx_points) > 0 else 0,
                        'bearing': calculate_initial_compass_bearing(
                            first_point=point,
                            second_point=gpx_points[-1]['gpx_point'],
                        ) if len(gpx_points) > 0 else 0,
                        'turn_angle': calculate_turn_angle(
                            bearing1=calculate_initial_compass_bearing(
                                first_point=gpx_points[-1]['gpx_point'],
                                second_point=gpx_points[-2]['gpx_point'],
                            ),
                            bearing2=calculate_initial_compass_bearing(
                                first_point=point,
                                second_point=gpx_points[-1]['gpx_point'],
                            ),
                        ) if len(gpx_points) > 1 else 0,
                    }
                )
    return pd.DataFrame(gpx_points)

In [4]:
gpx_df = load_gpx_todf(gpx_track)

In [65]:
gpx_df['smoothed_lat'] = gpx_df['latitude'].rolling(window=5,min_periods=1).mean()
gpx_df['smoothed_lon'] = gpx_df['longitude'].rolling(window=5,min_periods=1).mean()

In [77]:
for index, row in gpx_df.iterrows():
    if index == 0:
        continue
    gpx_df.loc[index, 'smoothed_bearing'] = calculate_initial_compass_bearing(
        first_point=GPXTrackPoint(latitude=gpx_df.loc[index-1, 'smoothed_lat'], longitude=gpx_df.loc[index-1, 'smoothed_lon']),
        second_point=GPXTrackPoint(latitude=gpx_df.loc[index, 'smoothed_lat'], longitude=gpx_df.loc[index, 'smoothed_lon']),
    )
    

In [78]:
for index, row in gpx_df.iterrows():
    if index == 0:
        continue
    gpx_df.loc[index, 'smoothed_turn_angle'] = calculate_turn_angle(
        bearing1=gpx_df.loc[index-1, 'smoothed_bearing'],
        bearing2=gpx_df.loc[index, 'smoothed_bearing'],
    )

In [33]:
gpx_df['turn_angle'] = gpx_df['turn_angle'].apply(
    lambda x: x if np.abs(x) < 45 else 45
)

In [57]:
import numpy as np

gpx_df['smoothed_turn_angle'] = gpx_df['turn_angle'].rolling(window=1,min_periods=1).mean()



In [79]:
from scipy.signal import argrelextrema


def identify_local_minmax(df, input_column_name, output_column_name='relative_extrema', order=None):
    # Initialize a new column 'turn' with NaN values
    df[output_column_name] = 0

    # Identify indices of local minima and maxima
    if not order:
        minima_indices = argrelextrema(df[input_column_name].values, np.less_equal)[0]
        maxima_indices = argrelextrema(df[input_column_name].values, np.greater_equal)[0]
    else:
        minima_indices = argrelextrema(df[input_column_name].values, np.less_equal, order=order)[0]
        maxima_indices = argrelextrema(df[input_column_name].values, np.greater_equal, order=order)[0]

    # Mark local minima and maxima in the 'turn' column
    df.loc[minima_indices, output_column_name] = df.loc[minima_indices, input_column_name]
    df.loc[maxima_indices, output_column_name] = df.loc[maxima_indices, input_column_name]

    return df

In [86]:
gpx_df = identify_local_minmax(gpx_df, input_column_name='smoothed_turn_angle', output_column_name='turn_extrema', order=30)
gpx_df = identify_local_minmax(gpx_df, input_column_name='elevation', output_column_name='elevation_extrema')
gpx_df['turn_direction'] = np.sign(gpx_df['turn_extrema'])

/var/folders/ks/7zd0y9gn3b1f5y7zw35zrm9r0000gq/T/ipykernel_61781/1385703926.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-20.47854641 -17.87053644  -5.30636298  -6.31073273 -11.37319377
 -31.72452214  -4.5454669  -13.36399875 -15.97676718  -2.33419101
  -8.84822863  -5.37631917  -3.88965641  -8.25836297  -2.46352887
 -10.35887104 -13.42011732  -3.15362907  -9.7364061   -2.86810407
  -8.94369477 -10.47640558  -8.7440737  -14.76279648  -4.05484389
  -5.76245114  -3.86700529  -2.55126711  -2.80351878  -1.99831221
  -3.22540164  -3.09995346  -3.49125279  -2.15589651  -2.44239716
  -3.48459237  -4.7449561  -11.10157342  -3.30415879  -3.20992364
  -7.76245188  -4.19042898  -4.11993083  -2.70896579  -1.84251301
  -5.04403322 -22.43028426 -14.15522314 -20.60408394  -4.01367514
  -2.09213314  -4.6360628  -10.34787456  -8.25206853 -23.17085939
 -14.82103469]' has dtype incompatible with int64, please explicitly cas

In [87]:
gpx_df

,gpx_point,latitude,longitude,elevation,time,distance,time_diff_s,elevation_diff,bearing,turn_angle,smoothed_turn_angle,turn_extrema,elevation_extrema,turn_direction,smoothed_lat,smoothed_lon,smoothed_bearing
0,"[trkpt:45.736284,7.315084@550.838341@2023-11-2...",45.736284,7.315084,550.838341,2023-11-20 17:02:16+00:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,550.838341,0.0,45.736284,7.315084,NaN
1,"[trkpt:45.73628,7.315069@550.838341@2023-11-20...",45.736280,7.315069,550.838341,2023-11-20 17:02:17+00:00,1.249245,1.0,0.0,69.089899,0.000000,NaN,0.000000,550.838341,0.0,45.736282,7.315076,249.089907
2,"[trkpt:45.736266,7.315043@550.838341@2023-11-2...",45.736266,7.315043,550.838341,2023-11-20 17:02:18+00:00,2.552687,1.0,0.0,52.350637,-16.739262,-13.473547,0.000000,550.838341,0.0,45.736277,7.315065,235.616360
3,"[trkpt:45.73625,7.315014@550.838341@2023-11-20...",45.736250,7.315014,550.838341,2023-11-20 17:02:19+00:00,2.873486,1.0,0.0,51.674378,-0.676260,-2.275998,0.000000,550.838341,0.0,45.736270,7.315053,233.340362
4,"[trkpt:45.736222,7.31498@550.838341@2023-11-20...",45.736222,7.314980,550.838341,2023-11-20 17:02:20+00:00,4.085058,1.0,0.0,40.282128,-11.392250,-6.828589,0.000000,550.838341,0.0,45.736260,7.315038,226.511773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704,"[trkpt:45.735716,7.315114@551.559318@2023-11-2...",45.735716,7.315114,551.559318,2023-11-20 18:04:00+00:00,1.336023,1.0,0.0,183.328807,-20.976426,-6.730671,0.000000,551.559318,0.0,45.735687,7.315099,30.996764
3705,"[trkpt:45.735729,7.315104@551.559318@2023-11-2...",45.735729,7.315104,551.559318,2023-11-20 18:04:01+00:00,1.641191,1.0,0.0,151.768682,-31.560125,-14.201662,0.000000,551.559318,0.0,45.735701,7.315105,16.795101
3706,"[trkpt:45.735752,7.315098@551.559318@2023-11-2...",45.735752,7.315098,551.559318,2023-11-20 18:04:02+00:00,2.598665,1.0,0.0,169.680698,17.912016,-14.821035,-14.821035,551.559318,-1.0,45.735718,7.315106,1.974067
3707,"[trkpt:45.735774,7.315096@551.559318@2023-11-2...",45.735774,7.315096,551.559318,2023-11-20 18:04:03+00:00,2.450165,1.0,0.0,176.369353,6.688655,-4.729917,0.000000,551.559318,0.0,45.735735,7.315105,357.244150


In [88]:
import plotly.express as px


In [89]:
import plotly.graph_objects as go

scatter = go.Scatter(
    x=gpx_df['time'],
    y=gpx_df['elevation'],
    mode='lines',
)

trace2 = go.Scatter(
    x=gpx_df[gpx_df['elevation_extrema'] != 0]['time'],
    y=gpx_df[gpx_df['elevation_extrema'] != 0]['elevation_extrema'],
    mode='markers',
)

fig = go.Figure(data=[scatter, trace2])
fig.show()

In [90]:
scatter = go.Scatter(
    x=gpx_df['time'],
    y=gpx_df['smoothed_turn_angle'],
    mode='lines',
)

trace2 = go.Scatter(
    x=gpx_df[gpx_df['turn_extrema'] != 0]['time'],
    y=gpx_df[gpx_df['turn_extrema'] != 0]['turn_extrema'],
    mode='markers',
)

fig = go.Figure(data=[scatter, trace2])
fig.show()


In [91]:

fig = px.scatter_mapbox(
    gpx_df[:3000],
    lat="latitude",
    lon="longitude",
    hover_name="time_diff_s",
    hover_data=['smoothed_turn_angle', "turn_angle"],
    color="turn_direction",
    zoom=15,
    width=1200,
    height=700,
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()